In [ ]:
## EU-DV-2 Threshold definition  – time-series
# The interface allow the expert user to define a threshold and select a 
# pollutant, a time window and a province and visualize a time series showing which days the 
# threshold has been exceeded and a percentage (exceeded days/total days).

# - button select a pollutant
# - button select threshold
# - button select start day
# - button select end day
# - button select start mounth
# - button select end mounth
# - button select start year
# - button select end year
# - button select a province 


In [59]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Output dinamici
output1 = widgets.Output()
output2 = widgets.Output()
output_map = widgets.Output()

# Funzione per recuperare le geometrie delle province (es. Lombardia)
def get_province_shape():
    url = "http://127.0.0.1:5000/api/province_shape"
    response = requests.get(url)
    data = response.json()
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry_province'] = gdf['geometry_province'].apply(wkt.loads)
    gdf.set_geometry('geometry_province', inplace=True)
    return gdf

# Funzione per generare mappa Folium centrata sulla Lombardia
def show_lombardia_map():
    with output_map:
        clear_output()
        m = folium.Map(location=[45.5, 9.5], zoom_start=8)
        try:
            prov = get_province_shape()
            folium.GeoJson(prov.__geo_interface__, name="Province").add_to(m)
        except Exception as e:
            print("Errore caricando le province:", e)
        display(m)

# Dropdown inquinanti
dropdown = widgets.Dropdown(
    options=[
        "Ammoniaca", "Arsenico", "Benzo(a)pirene", "Benzene", "Biossido di Azoto",
        "Biossido di Zolfo", "BlackCarbon", "Cadmio", "Monossido di Azoto",
        "Monossido di Carbonio", "Nikel", "Ossidi di Azoto", "Ozono", "PM10",
        "PM10 (SM2005)", "Particelle sospese PM2.5", "Particolato Totale Sospeso",
        "Piombo"
    ],
    description='Inquinante:'
)

# Funzione quando cambia dropdown: mostra slider e collega evento
def on_dropdown_change1(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output1:
            clear_output()
            slider = widgets.IntSlider(
                value=0,
                min=0,
                max=80,
                step=1,
                description=f'Soglia ({change["new"]}):',
                continuous_update=False
            )
            display(slider)

            def on_slider_change(slider_change):
                if slider_change['type'] == 'change' and slider_change['name'] == 'value':
                    with output2:
                        clear_output()

                        start_date = widgets.DatePicker(description='Inizio:')
                        end_date = widgets.DatePicker(description='Fine:')

                        # Funzione per controllare se entrambe le date sono selezionate
                        def check_dates(_):
                            if start_date.value and end_date.value:
                                show_lombardia_map()

                        # Osserva cambiamenti di entrambe le date
                        start_date.observe(check_dates, names='value')
                        end_date.observe(check_dates, names='value')

                        display(widgets.VBox([start_date, end_date]))

            slider.observe(on_slider_change, names='value')

# Collegamento eventi
dropdown.observe(on_dropdown_change1, names='value')

# Visualizzazione finale
display(dropdown)
display(output1)
display(output2)
display(output_map)


Dropdown(description='Inquinante:', options=('Ammoniaca', 'Arsenico', 'Benzo(a)pirene', 'Benzene', 'Biossido d…

Output()

Output()

Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium
import requests
import geopandas as gpd
from shapely import wkt

# Output per la mappa
output_map = widgets.Output()

# Recupera le geometrie delle province
def get_province_shape():
    url = "http://127.0.0.1:5000/api/province_shape"
    response = requests.get(url)
    data = response.json()
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)
    return gdf

# Caricamento delle province e creazione dropdown
prov_gdf = get_province_shape()
province_names = sorted(prov_gdf['nome_provincia'].unique().tolist())
dropdown_prov = widgets.Dropdown(options=province_names, description="Provincia:")

# Funzione per mostrare la mappa aggiornata in base alla provincia selezionata
def show_lombardia_map(selected_prov):
    with output_map:
        clear_output()
        # Usa tiles='OpenStreetMap' per forzare stile OSM
        m = folium.Map(location=[45.5, 9.5], zoom_start=8, tiles='OpenStreetMap')

        try:
            selected_geom = prov_gdf[prov_gdf['nome_provincia'] == selected_prov]
            others = prov_gdf[prov_gdf['nome_provincia'] != selected_prov]

            # Provincia selezionata in rosso
            folium.GeoJson(
                selected_geom.__geo_interface__,
                name="Provincia selezionata",
                style_function=lambda x: {'color': 'red', 'fillOpacity': 0.4, 'weight': 3}
            ).add_to(m)

            # Altre province in grigio
            folium.GeoJson(
                others.__geo_interface__,
                name="Altre province",
                style_function=lambda x: {'color': 'gray', 'fillOpacity': 0.1, 'weight': 1}
            ).add_to(m)

            folium.LayerControl().add_to(m)
        except Exception as e:
            print("Errore caricando le province:", e)

        display(m)

# Callback: aggiorna la mappa quando cambia la selezione
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        show_lombardia_map(change['new'])

dropdown_prov.observe(on_dropdown_change)

# Layout a due colonne simulando dropdown "nella mappa"
display(
    widgets.HBox([
        widgets.VBox([widgets.HTML("<b>Seleziona provincia:</b>"), dropdown_prov]),
        output_map
    ])
)

# Mostra la mappa iniziale
show_lombardia_map(province_names[0])

In [ ]:
# EU-DV-2 

data = {"var_pollutant": "Benzene", "var_start_date" :"2024-12-19 18:00:00", "var_end_date" :"2025-01-01 00:00:00","var_province":"MILANO"} #Right now data is a dictionary
t=requests.post(url="http://127.0.0.1:5000/api/EU_DV_2", json=data) 

try:
    data = t.json() #This will convert the response to a json object
    print("The query returned", len(data), "rows")
    print(type(data))
except requests.exceptions.JSONDecodeError:
    print("Risposta non valida JSON!")
    print("Contenuto della risposta:", t.text)
    data = None

df = pd.DataFrame(data)
print(df)